## Signal Amplification by Reversible Exchange (SABRE) - Pyridine

This example builds upon the `sabre_pyridine_simple.ipynb` by providing a more advanced demonstration of SABRE simulations. This example incorporates the relaxation and chemical exchange to the simulation. Key details of the simulation are as follows:

- **Spin System**: The system consists of three parts:
    - SABRE Complex: Two hydride protons and one equatorial pyridine ligand (all NMR-active nuclei included).
    - Free Ligand: Pyridine molecules in the solution (all NMR-active nuclei included).
    - Parahydrogen
- **Simplifications**: The basis set is truncated to include spin orders up to 4.
- **Simulation Conditions**: The simulation is performed near the energy level anticrossing field, leading to coherent oscillations between quantum states. This should result in close to optimal polarization transfer.
- **Performance**: The example runs in approximately 15 minutes on a laptop with an 11th-generation i5 processor and 16 GB ram.

### Imports

- **NumPy**: Used for creating and manipulating arrays.
- **Matplotlib**: Used for plotting data and visualizing results.
- **Spinguin**: Provides the core functionality for simulating spin systems, including Hamiltonian and relaxation superoperator calculations, propagators, and measurements.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import spinguin as sg

### Simulation Parameters

The following simulation parameters are defined:

- **Magnetic Field (T)**: This is a global parameter in the Spinguin package. It is set to 5.3 mT.
- **Temperature (K)**: This is also a global parameter in the Spinguin pacakge. It is set to 295 K (room temperature).
- **Time Step (s)**: Set to 2 ms.
- **Number of Time Steps**: Set to 30 000, resulting in a total simulation time of 60 seconds.

In [ ]:
# Define global simulation parameters
sg.parameters.magnetic_field = 5.3e-3
sg.parameters.temperature = 295

# Define the evolution parameters
dt = 2e-3
N_steps = 30000

### Define the Spin Systems

In this example, we define three spin systems:
- **SABRE Complex**: Hydride protons and all NMR-active nuclei of one equatorial pyridine ligand.
- **Free Ligand**: Pyridine molecules in solution.
- **Parahydrogen**

For each of the spin systems, the following is set:
- **Spin System**: First, the spin system is initialized by defining the isotopes.
- **Maximum Spin Order**: The simulations are performed in Liouville space using the spherical tensor basis set. The full basis set scales as $4^n$ with the number of spins $n$, making the use of the full basis set impractical for large systems. Hence, the basis set is truncated to include spin orders up to 4 for the SABRE complex and the free pyridine ligand. For parahydrogen, no approximation is used.
- **Build the Basis Set**: Finally, the basis set is built using the `build()` method.

In [ ]:
# Spin system and basis set for the complex
spin_system_c = sg.SpinSystem(['1H', '1H', '1H', '1H', '1H', '1H', '1H', '14N'])
spin_system_c.basis.max_spin_order = 4
spin_system_c.basis.build()

# Spin system and basis set for the free ligand
spin_system_s = sg.SpinSystem(['1H', '1H', '1H', '1H', '1H', '14N'])
spin_system_s.basis.max_spin_order = 4
spin_system_s.basis.build()

# Spin system and basis set for the parahydrogen
spin_system_H2 = sg.SpinSystem(['1H', '1H'])
spin_system_H2.basis.max_spin_order = 2
spin_system_H2.basis.build()

### Define the NMR Interactions

This simulation incorporates both the coherent and incoherent NMR interactions.

Coherent interactions:
- **Zeeman Interaction**
- **Isotropic Chemical Shift**
- **Scalar J-Coupling**

Incoherent interactions:
- **Dipolar Coupling**
- **Anisotropic Chemical Shift**
- **Quadrupolar Coupling**
- **Scalar Relaxation of the Second Kind (SR2K)**

Simulation of these interactions involves the definition of the following spin system properties:

- **Chemical Shifts**: Specified in parts per million (ppm).
- **Scalar Couplings**: Specified in Hertz (Hz).
- **Molecular Coordinates**: Specified in Ångström (Å).
- **Shielding Tensors** Specified in parts per million (ppm).
- **Electric Field Gradient Tensors**: Specified in atomic units (a.u.).

These properties are set as attributes to the previously created `SpinSystem` object. Note that NMR interactions are not defined for the parahydrogen. This is because, in the experiments, fresh parahydrogen is constantly pushed to the system.

In [ ]:
# Define the NMR interactions for SABRE complex
spin_system_c.chemical_shifts = [-22.7, -22.7, 8.34, 8.34, 7.12, 7.12, 7.77, 43.60]
spin_system_c.J_couplings = [
    [ 0,     0,      0,      0,      0,      0,      0,     0],
    [-6.53,  0,      0,      0,      0,      0,      0,     0],
    [ 0.00,  1.66,   0,      0,      0,      0,      0,     0],
    [ 1.40,  0.00,  -0.06,   0,      0,      0,      0,     0],					
    [-0.09,	 0.35,	 6.03,	 0.14,	 0,      0,      0,     0],					
    [ 0.38, -0.13,	 0.09,	 5.93,	 0.06, 	 0,      0,     0],			
    [ 0.01,	 0.03,	 1.12,	-0.02,	 7.75,  -0.01, 	 0,     0],
    [-0.30,  15.91,  4.47,   0.04,   1.79,   0,     -0.46,  0]
]
spin_system_c.xyz = [
    [ 0.9649170,  1.2271534, -1.2031835],
    [ 1.9547078, -0.4342818, -0.1922623],
    [-2.5492743, -0.9988969, -0.2721286],
    [-1.3895773, -2.6746310, -1.2331514],
    [-4.5704762, -1.0068808, -1.6740361],
    [-1.5724294, -5.0099155, -0.4837575],
    [-4.5894708,  0.3577455, -3.7835019],
    [-1.4702745,  0.2327446, -1.5095832]
]
shielding_c = np.zeros((8, 3, 3))
shielding_c[7] = np.array([
    [-134.70, -123.93, -49.86],
    [-147.79,  64.47,   221.96],
    [-62.63,   223.57, -60.57]
])
spin_system_c.shielding = shielding_c
efg_c = np.zeros((8, 3, 3))
efg_c[7] = np.array([
    [-0.3426, -0.0417, -0.4514],
    [-0.0417,  0.3727,  0.1186],
    [-0.4514,  0.1186, -0.0301]
])
spin_system_c.efg = efg_c

# Define NMR interactions for the free ligand
spin_system_s.chemical_shifts = [8.56, 8.56, 7.47, 7.47, 7.88, 95.94]
spin_system_s.J_couplings = [
    [ 0,     0,      0,      0,      0,      0],
    [-1.04,  0,      0,      0,      0,      0],
    [ 4.85,  1.05,   0,      0,      0,      0],
    [ 1.05,  4.85,   0.71,   0,      0,      0],
    [ 1.24,  1.24,   7.55,   7.55,   0,      0],
    [ 8.16,  8.16,   0.87,   0.87,  -0.19,   0]
]
spin_system_s.xyz = [
    [ 2.0495335, 0.0000000, -1.4916842],
    [-2.0495335, 0.0000000, -1.4916842],
    [ 2.1458878, 0.0000000,  0.9846086],
    [-2.1458878, 0.0000000,  0.9846086],
    [ 0.0000000, 0.0000000,  2.2681296],
    [ 0.0000000, 0.0000000, -1.5987077]
]
shielding_s = np.zeros((6, 3, 3))
shielding_s[5] = np.array([
    [-406.20, 0.00,   0.00],
    [ 0.00,   299.44, 0.00],
    [ 0.00,   0.00,  -181.07]
])
spin_system_s.shielding = shielding_s
efg_s = np.zeros((6, 3, 3)) 
efg_s[5] = np.array([
    [0.3069, 0.0000,  0.0000],
    [0.0000, 0.7969,  0.0000],
    [0.0000, 0.0000, -1.1037]
])
spin_system_s.efg = efg_s

## Define the Relaxation Properties

Simulation of Redfield theory requires the definition of the following properties for relaxation:
- **Relaxation Theory**: Set to `"redfield"`.
- **Rotational Correlation Time (ps)**: Characteristic time for the molecular tumbling.
- **Scalar Relaxation of the Second Kind (SR2K)**: Processing the SR2K is enabled.
- **Thermalization**: Thermalization of the relaxation superoperator using Levitt-di Bari method is enabled.

These properties are set under the relaxation attribute of each of the spin systems.

In [ ]:
# Define the relaxation properties for the SABRE complex
spin_system_c.relaxation.theory = "redfield"
spin_system_c.relaxation.tau_c = 43e-12
spin_system_c.relaxation.sr2k = True
spin_system_c.relaxation.thermalization = True

# Define the relaxation properties for the free ligand
spin_system_s.relaxation.theory = "redfield"
spin_system_s.relaxation.tau_c = 5.7e-12
spin_system_s.relaxation.sr2k = True
spin_system_s.relaxation.thermalization = True

## Define the Chemical Exchange Properties

Simulation of the chemical exchange in SABRE involves the following processes:
- **Dissociation of the Complex**: The SABRE-complex dissociates into free hydrogen gas and pyridine molecule.
- **Formation of the Complex**: Two possible association reactions may take place:
    - Association of old $\mathrm{H}_2$ and substrate
    - Association of new $\mathrm{H}_2$ (parahydrogen) and substrate

To simulate these processes, the following has to be defined:
- **Dissociation Rates (1/s)**: The rates that describe how often a substrate or hydrogen molecule dissociates.
- **Concentrations (mol/l)**: Concentrations of the SABRE-complex and free ligand in the solution.
- **Spin Maps**: Specifies how the association / dissociation is indexed. The hydride protons are at indices `[0, 1]` and the pyridine molecule at indices `[2, 3, 4, 5, 6, 7]`.
- **Permutation Map**: When the pyridine molecule is bound to the complex, the nuclei in the pyridine molecule are not symmetric. However, this is not true after dissociation - the two ortho protons and meta protons are both chemically and magnetically equivalent. This is taken into account using a permutation map.

In [ ]:
# Define the dissociation rates
k_H2 = 1.6
k_s = 10

# Define the concentrations
c_c = 0.0005
c_s = 0.0135

# Define the spin maps
spin_map_s = [2, 3, 4, 5, 6, 7]
spin_map_H2 = [0, 1]

# Define the permutation map for pyridine
perm_map = [1, 0, 3, 2, 4, 5]

## Assemble the total Liouvillians

The following steps are performed in this section:

- **Hamiltonian**: The Hamiltonian is calculated for the defined spin systems.
- **Relaxation**: The relaxation superoperator is calculated for the defined spin systems.
- **Liouvillian**: In the Spinguin package, the Liouvillian is defined as $L = -iH - R + K$. We use the inbuilt function to calculate the Liouvillian using the correct definition. Note that $K$ is not given, as the chemical exchange in SABRE is non-linear. Therefore, the exchange has to be performed manually during each time step.

In [ ]:
# Calculate the Hamiltonians
H_c = sg.hamiltonian(spin_system_c)
H_s = sg.hamiltonian(spin_system_s)

# Calculate the relaxation superoperators
R_c = sg.relaxation(spin_system_c)
R_s = sg.relaxation(spin_system_s)

# Calculate the Liouvillians
L_c = sg.liouvillian(H_c, R_c)
L_s = sg.liouvillian(H_s, R_s)

## Move to Zero-Quantum Subspace

The initial state in the SABRE simulations involves only product operators that lie in the zero-quantum subspace. In addition, the defined Liouvillian does not mix the zero-quantum subspace with the other coherence orders. Therefore, the basis set can be truncated to involve only the zero-quantum terms. This involves:
- **Truncating the Basis Set**: Product operators whose coherence order is not equal to zero are removed from the basis set.
- **Transforming the Liouvillian**: The Liouvillian, which was previously generated, has to be transformed into the new truncated basis set.

These two steps are performed simultaneously by calling the `truncate_by_coherence()` method from the basis set.

In [ ]:
# Truncate the basis set and transform the Liouvillian
L_c = spin_system_c.basis.truncate_by_coherence([0], L_c)
L_s = spin_system_s.basis.truncate_by_coherence([0], L_s)

## Calculate the Time Propagators

Now, when the basis set has been truncated, it is computationally less expensive to compute the time propagator. Next, we compute that using the truncated Liouvillian and the previously defined time step.

In [ ]:
# Calculate the time propagators
P_c = sg.propagator(L_c, dt)
P_s = sg.propagator(L_s, dt)

### Assign the Initial States

Thermal equilibrium is assigned for the SABRE-complex and the free ligand. Parahydrogen is created by assigning a singlet state.

In [ ]:
# Make the initial states
rho_c = sg.equilibrium_state(spin_system_c)
rho_s = sg.equilibrium_state(spin_system_s)
rho_H2 = sg.singlet_state(spin_system_H2, 0, 1)

### Create an Empty Array for Storing Magnetizations

To store the magnetizations during the simulation, we need to create an empty array. In this simulation, we store the magnetizations only from the free ligand.

- The simulation involves 30 000 time steps.
- After each time step, the magnetization of each spin will be calculated.
- The array will store the magnetizations for all spins at each time step.

In [ ]:
# Create an empty array to store magnetizations
magnetizations = np.empty((spin_system_s.nspins, N_steps), dtype=complex)

### Perform the Time Evolution

- Loop over the defined number of time steps.
- Calculate the magnetization for each spin at each time step.
- Save the calculated magnetizations to the previously created array for later analysis.
- Perform the chemical exchange.
- Propagate the spin systems forward in time during each step using the time propagator.

In [ ]:
# Evolve the system for the specified number of steps
for step in range(N_steps):

    # Measure the magnetization for each spin
    for spin in range(spin_system_s.nspins):
        magnetizations[spin, step] = sg.measure(spin_system_s, rho_s, f'I(z, {spin})')

    # Dissociation of substrate
    rho_H2_old, rho_s_old = sg.dissociate(spin_system_H2, spin_system_s, spin_system_c, rho_c, spin_map_H2, spin_map_s)

    # Calculate the complex where only the substrate is exchanged
    rho_c_new_s_old_H2 = sg.associate(spin_system_H2, spin_system_s, spin_system_c, rho_H2_old, rho_s, spin_map_H2, spin_map_s)

    # Calculate the complex where both the substrate and H2 are exchanged
    rho_c_new_s_new_H2 = sg.associate(spin_system_H2, spin_system_s, spin_system_c, rho_H2, rho_s, spin_map_H2, spin_map_s)

    # Account for the symmetry of pyridine
    rho_s_old = (rho_s_old + sg.permute_spins(spin_system_s, rho_s_old, perm_map)) / 2

    # Exchange process for free substrate
    rho_s = rho_s + c_c / c_s * dt * k_s * (rho_s_old - rho_s)

    # Exchange process for the complex
    rho_c = rho_c + dt * (
        (k_s - k_H2) * (rho_c_new_s_old_H2 - rho_c) +
        k_H2 * (rho_c_new_s_new_H2 - rho_c)
    )

    # Propagate the system forward in time
    rho_c = P_c @ rho_c
    rho_s = P_s @ rho_s

### Plot the Magnetizations and Visualize the Results

- Calculate the time axis for the simulation.
- Plot the magnetizations of each spin as a function of time.
- Observe that, in contrast to the simple SABRE example, coherent oscillation is no longer seen. Instead, we find a nearly single-exponential build-up of the polarization of the substrate protons. In addition, we find that the quadrupolar nitrogen-14 is not becoming hyperpolarized.

In [ ]:
# Create a time axis for the simulation
t = sg.time_axis(N_steps, dt)

# Plot the magnetizations for each spin as a function of time
for spin in range(spin_system_s.nspins):
    plt.plot(t, np.real(magnetizations[spin]), label=f"Spin {spin+1}")

# Add a legend to identify each spin
plt.legend(loc="upper right")

# Add labels and title to the plot for clarity
plt.xlabel("Time (s)")
plt.ylabel("Magnetization")
plt.title("SABRE-Hyperpolarization of Pyridine")

# Adjust layout to prevent overlapping elements and display the plot
plt.tight_layout()
plt.show()

# Clear the figure to avoid overlapping plots in subsequent cells
plt.clf()